# Decorator Notes

These are notes based on Real Python's [primer](https://realpython.com/primer-on-python-decorators/#functions).

## Simple Decorators

### Returning Functions

In [1]:
def parent(num):
    def first_child():
        return "Hi, I am Emma"

    def second_child():
        return "Call me Liam"

    if num == 1:
        return first_child
    else:
        return second_child

In [2]:
parent(2)

<function __main__.parent.<locals>.second_child()>

### Simple Decorators with Syntactic Sugar

In [3]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

def say_whee():
    print("Whee!")

say_whee = my_decorator(say_whee)

In [4]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


Now with syntactic sugar:

In [5]:
def my_decorator(func):
    def wrapper():
        print("Something is happening before the function is called.")
        func()
        print("Something is happening after the function is called.")
    return wrapper

@my_decorator
def say_whee():
    print("Whee!")

In [6]:
say_whee()

Something is happening before the function is called.
Whee!
Something is happening after the function is called.


### Decorating Functions with Arguments

Decorating simple functions is easy enough, but what happens when decorated function takes in arguments? The second function `greet()` is incompatible with the decorator, `do_twice` since the inner function does not take any arguments.

In [17]:
def do_twice(func):
    def wrapper_do_twice():
        func()
        func()
    return wrapper_do_twice

@do_twice
def say_whee():
    print('Whee!')

@do_twice
def greet(name: str):
    print(f'Hello {name}!')

say_whee()

Whee!
Whee!


In [18]:
greet('Jonathan')

TypeError: do_twice.<locals>.wrapper_do_twice() takes 0 positional arguments but 1 was given

Instead, we can add arguments to our decorator to handle any potential parameters using `*args, **kwargs**`:

In [33]:
def do_twice(func):
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")
    
@do_twice
def greet(name):
    print(f"Hello {name}")

In [34]:
say_whee()

Whee!
Whee!


In [35]:
greet('Jonathan')

Hello Jonathan
Hello Jonathan


### Decorating Functions with Return Values

For functions that return values, we must make sure the decorator's inner function also returns a value. If not, like in the case of the `do_twice` decorator's current form, the decorated function's return value will get lost: 

In [37]:
@do_twice
def return_greeting(name: str):
    print('Creating greeting')
    return f'Hi {name}!'

hi_jonathan = return_greeting('Jonathan')

Creating greeting
Creating greeting


In [39]:
print(hi_jonathan)

None


In [30]:
def do_twice(func):
    def do_twice_wrapper(*args, **kwargs):
       func(*args, **kwargs)
       return func(*args, **kwargs)
    return do_twice_wrapper

@do_twice
def return_greeting(name: str):
    print('Creating greeting')
    return f'Hi {name}!'

hi_jonathan = return_greeting('Jonathan')

Creating greeting
Creating greeting


In [31]:
print(hi_jonathan)

Hi Jonathan!


### Introspection 

Python functions have built-in documentation. Decorated ones simply show themselves as the inner function of the wrapper. To get the original, useful documenation of a decorated function, the decorator must have `@functools.wraps(func)` above its inner function definition:

In [40]:
print

<function print(*args, sep=' ', end='\n', file=None, flush=False)>

In [20]:
print.__name__

'print'

In [21]:
help(print)

Help on built-in function print in module builtins:

print(...)
    print(value, ..., sep=' ', end='\n', file=sys.stdout, flush=False)
    
    Prints the values to a stream, or to sys.stdout by default.
    Optional keyword arguments:
    file:  a file-like object (stream); defaults to the current sys.stdout.
    sep:   string inserted between values, default a space.
    end:   string appended after the last value, default a newline.
    flush: whether to forcibly flush the stream.



In [22]:
say_whee

<function __main__.do_twice.<locals>.wrapper_do_twice(*args, **kwargs)>

In [23]:
say_whee.__name__

'wrapper_do_twice'

In [24]:
help(say_whee)

Help on function wrapper_do_twice in module __main__:

wrapper_do_twice(*args, **kwargs)



In [31]:
from functools import wraps

def do_twice(func):
    @wraps(func)
    def wrapper_do_twice(*args, **kwargs):
        func(*args, **kwargs)
        return func(*args, **kwargs)
    return wrapper_do_twice

@do_twice
def say_whee():
    print("Whee!")


In [32]:
say_whee

<function __main__.say_whee()>

In [33]:
say_whee.__name__

'say_whee'

In [34]:
help(say_whee)

Help on function say_whee in module __main__:

say_whee()



In [ ]:
hasattr()

## Decorator Real World Examples

The pattern for many decorators will take this form:

In [41]:
import functools

def decorator(func):
    @functools.wrap(func)
    def wrapper_decorator(*args, **kwargs):
        # Do something before
        value = func(*args, **kwargs)
        # Do something after
        return value
    return wrapper_decorator

### Timing Functions

In [45]:
import functools
import time

def timer(func):
    """Print the runtime of the decorated function"""
    @functools.wraps(func)
    def wrapper_timer(*args, **kwargs):
        start = time.perf_counter()
        value = func(*args, **kwargs)
        end = time.perf_counter()
        runtime = end - start
        print(f"Finished {func.__name__}() in {runtime:.4f} secs")
        return value
    return wrapper_timer

In [47]:
@timer
def waste_some_time(num_times: int):
    for _ in range(num_times):
        sum([number**2 for number in range(10_000)])

waste_some_time(1)
        

Finished waste_some_time() in 0.0021 secs


In [48]:
waste_some_time(999)

Finished waste_some_time() in 1.1273 secs


### Debugging Code

We can take advantage of the namespace access in the wrapped function to display parameters and return values for debugging:

In [52]:
import functools

def debug(func):
    """Print the function signature and return value"""
    @functools.wraps(func)
    def wrapper_debug(*args, **kwargs):
        args_repr = [repr(a) for a in args]
        kwargs_repr = [f"{k}={repr(v)}" for k, v in kwargs.items()]
        signature = ", ".join(args_repr + kwargs_repr)
        print(f"Calling {func.__name__}({signature})")
        value = func(*args, **kwargs)
        print(f"{func.__name__}() returned {repr(value)}")
        return value
    return wrapper_debug

@debug
def make_greeting(name: str, age: int=None):
    if age is None:
        return f"Howdy {name}!"
    else:
        return f"Whoa {name}! {age} already?! You're growing up!"

make_greeting("Benjamin")

Calling make_greeting('Benjamin')
make_greeting() returned 'Howdy Benjamin!'


'Howdy Benjamin!'

In [53]:
make_greeting("Juan", age=114)

Calling make_greeting('Juan', age=114)
make_greeting() returned "Whoa Juan! 114 already?! You're growing up!"


"Whoa Juan! 114 already?! You're growing up!"

This is more impactful when used on small inconvenience functions not called directly:

In [54]:
import math

math.factorial = debug(math.factorial)

def approximate_e(terms=18):
    return sum(1 / math.factorial(n) for n in range(terms))

approximate_e(terms=5)

Calling factorial(0)
factorial() returned 1
Calling factorial(1)
factorial() returned 1
Calling factorial(2)
factorial() returned 2
Calling factorial(3)
factorial() returned 6
Calling factorial(4)
factorial() returned 24


2.7083333333333335

### Slowing Down Code

Sometimes, it's useful to slow down code for the purposes of rate limiting, which can be achieved by a decorator:

In [55]:
import functools
import time

def slow_down(func):
    @functools.wraps(func)
    def wrapper_slow_down(*args, **kwargs):
        time.sleep(1)
        return func(*args, **kwargs)
    return wrapper_slow_down

@slow_down
def countdown(from_number):
    if from_number < 1:
        print("Liftoff!")
    else:
        print(from_number)
        countdown(from_number - 1)

countdown(3)

3
2
1
Liftoff!


### Registering Plugins

Here we use a decorator to leave functions unmodified, but record them in a dictionary. This is a list separate from `globals()`:

In [56]:
PLUGINS = dict()

def register(func):
    """Register a funciton as a plug-in"""
    PLUGINS[func.__name__] = func
    return func

@register
def say_hello(name: str):
    return f"Hello {name}"

@register
def be_awesome(name: str):
    return f"Yo {name}, together, we're awesome!"

PLUGINS

{'say_hello': <function __main__.say_hello(name: str)>,
 'be_awesome': <function __main__.be_awesome(name: str)>}

In [58]:
be_awesome('Jonathan')

"Yo Jonathan, together, we're awesome!"

Here we use the string representation of a Python object using the `!r` flag.

In [59]:
import random

def randomly_greet(name):
    greeter, greeter_func = random.choice(list(PLUGINS.items()))
    print(f"Using {greeter!r}")
    return greeter_func(name)

randomly_greet("Alice")

Using 'say_hello'


'Hello Alice'

### Authenticating Users

In [ ]:
import functools
from flask import Flask, g, request, redirect, url_for

app = Flask(__name__)

def login_required(func):
    """Make sure user is logged in before proceeding"""
    @functools.wraps(func)
    def wrapper_login_required(*args, **kwargs):
        if g.user is None:
            return redirect(url_for("login", next=request.url))
        return func(*args, **kwargs)
    return wrapper_login_required

@app.route("/secret")
@login_required
def secret():
    ...

## Fancy Decorators

### Decorating Classes

This can be achieved on the individual methods or the entire class itself:

In [64]:
class TimeWaster:
    @debug
    def __init__(self, max_num: int):
        self.max_num = max_num

    @timer
    def waste_time(self, num_times: int):
        for _ in range(num_times):
            sum([number**2 for number in range(self.max_num)])

tw = TimeWaster(1000)

Calling __init__(<__main__.TimeWaster object at 0x000001D0871DACF0>, 1000)
__init__() returned None


In [65]:
tw.waste_time(999)

Finished waste_time() in 0.1318 secs


In [ ]:
from dataclasses import dataclass

@dataclass
class PlayingCard:
    rank: str
    suit: str


Decorating a class does not decorate its methods:

In [66]:
@timer
class TimeWaster:
    def __init__(self, max_num):
        self.max_num = max_num

    def waste_time(self, num_times):
        for _ in range(num_times):
            sum([i**2 for i in range(self.max_num)])

tw = TimeWaster(1000)

Finished TimeWaster() in 0.0000 secs


In [68]:
tw.waste_time(999)

### Nesting Decorators

In [70]:
@debug
@do_twice
def greet(name):
    print(f'Hello {name}!')

greet('Yadi')

Calling wrapper_do_twice('Yadi')
Hello Yadi!
Hello Yadi!
wrapper_do_twice() returned None


In [71]:
@do_twice
@debug
def greet(name):
    print(f'Hello {name}!')

greet('Yadi')

Calling greet('Yadi')
Hello Yadi!
greet() returned None
Calling greet('Yadi')
Hello Yadi!
greet() returned None


### Defining Decorators with Arguments

We know how to define a decorator on general functions and how to add functionality to the decorated function. To add parameters to a decorator, we apply the same principle and *decorate the decorator*.


This adds another layer. The syntactic sugar saves more typing since another layer of nesting requires another function reassignment, from outer to inner.

In [74]:
import functools

def repeat(num_times: int):
    def decorator_repeat(func):
        functools.wraps(func)
        def wrapper_decorator_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_decorator_repeat
    return decorator_repeat

def greet(name: str):
    print(f'Hello {name}!')

repeat4 = repeat(4)
greet = repeat4(greet)
greet('Jonathan')

Hello Jonathan!
Hello Jonathan!
Hello Jonathan!
Hello Jonathan!


In [76]:
@repeat(4)
def greet(name: str):
    print(f'Hello {name}!')

greet('Jonathan')

Hello Jonathan!
Hello Jonathan!
Hello Jonathan!
Hello Jonathan!


### Creating Decorators with Optional Arguments

To modify our decorator to have optional arguments, we must modify the outermost layer in two ways:
- to take in arguments with the  * syntax
- control flow of optional parameter values 


This approach wraps the decorator with an outer layer that takes optional arguments, but also allows for a "skip" connection to layer below in case parameters are not specified. In this case, the synactic sugar will only take in the first argument, which is the function to be decorated.

In [ ]:
def name(_func=None, *, key1=value1, key2=value2, ...):
    def decorator_name(func):
        ... # Create and return a wrapper function

    if _func is None:
        return decorator_name
    else:
        return decorator_name(_func)

In [78]:
import functools

def repeat(_func=None, *, num_times=2):
    def decorator_repeat(func):
        @functools.wraps(func)
        def wrapper_repeat(*args, **kwargs):
            for _ in range(num_times):
                value = func(*args, **kwargs)
            return value
        return wrapper_repeat

    if _func is None:
        return decorator_repeat
    else:
        return decorator_repeat(_func)

@repeat
def say_whee():
    print('Whee!')

@repeat(num_times=3)
def greet(name):
    print(f'Hello {name}!')

say_whee()

Whee!
Whee!


In [79]:
greet('Jonathan')

Hello Jonathan!
Hello Jonathan!
Hello Jonathan!


### Tracking State in Decorators

Here we an use *function attributes* to track a state variable such as a counter

In [ ]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f'Call {wrapper_count_calls.num_calls} of {func.__name__}()')
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls
    

### Using Classes as Decorators

If we step back and think about how a closure decorator works, we have the following steps:
- if there are decorator parameters, first initialize an "instance" with them set 
- next, rename the function to be modified with its wrapped self: `func = decorator(func)`
- finally, call the function as normal with its usual inputs


These steps are similar to how a class instance works with a `__init__()` and `__call__()` method.
The analogous steps are: 
- define an `__init__()` that sets the parameters of the decorator, acting as the outermost function in a nested closure
- define a `__call__()` method with the structure of a closure
    - use `functools.wraps()`  for appropriate introspection
    - have the `__call__()` method take in the function as an argument
    - define an inner function matching the decorated function's inputs along with additional logic 
    - return the inner function to finalize the `__call__()` method
- for simpler, unparameterized decorating classes, use `functools.update_wrapper()` instead of `functools.wraps()` in the `__init__()` method for proper introspection

In [80]:
class Counter:
    def __init__(self, start=0):
        self.count = start
    def __call__(self):
        self.count += 1
        print(f"Current count is {self.count}")

counter = Counter()

counter()
    

Current count is 1


In [81]:
counter()

Current count is 2


Now, with the modifications to make it a simple decorator without parameters:

In [82]:
import functools
class Counter:
    def __init__(self, func):
        functools.update_wrapper(self, func)
        self.func = func
        self.num_calls = 0

    def __call__(self, *args, **kwargs):
        self.num_calls += 1
        print(f"Call {self.num_calls} of {self.func.__name__}()")
        return self.func(*args, **kwargs)

@Counter
def say_whee():
    print('Whee!')


say_whee()

Call 1 of say_whee()
Whee!


In [83]:
say_whee()

Call 2 of say_whee()
Whee!


In [84]:
say_whee.num_calls

2

Finally, we have this example that was generated by Gemini where the `__init__()` sets parameters and the `__call__()` is implemented as a closure with general paramters:

In [85]:
import functools
from datetime import datetime

class FunctionLogger:
    """
    A class-based decorator to log function execution details.
    It takes parameters to customize the log output.
    """
    def __init__(self, log_level="INFO", format_str="{timestamp} [{level}] - Entering '{func_name}'"):
        """
        The __init__ method is called when the decorator is instantiated.
        It captures the decorator's arguments.
        Example: @FunctionLogger(log_level="DEBUG")
        """
        print(f"Decorator `FunctionLogger` is being initialized with level: {log_level}")
        self.log_level = log_level
        self.format_str = format_str

    def __call__(self, func):
        """
        The __call__ method is invoked with the decorated function as its argument.
        It must return a replacement (wrapper) function.
        """
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            """
            This is the wrapper function that gets executed instead of the original.
            It contains the decorator's core logic.
            """
            timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            
            # 1. Logic before the original function is called
            log_message = self.format_str.format(
                timestamp=timestamp,
                level=self.log_level,
                func_name=func.__name__
            )
            print(log_message)
            
            # 2. Call the original function
            result = func(*args, **kwargs)
            
            # 3. Logic after the original function is called
            print(f"{timestamp} [{self.log_level}] - Exiting '{func.__name__}'")
            
            return result
            
        # The __call__ method returns the wrapper function
        return wrapper

# --- Example Usage ---

@FunctionLogger(log_level="DEBUG", format_str="{timestamp} - {level} - Calling `{func_name}`...")
def add(x, y):
    """This function adds two numbers together."""
    print(f"  > Inside add({x}, {y})")
    return x + y

@FunctionLogger(log_level="WARNING")
def greet(name):
    """This function greets a person."""
    print(f"  > Hello, {name}!")

greet('Jonathan')

Decorator `FunctionLogger` is being initialized with level: DEBUG
Decorator `FunctionLogger` is being initialized with level: WARNING
2025-09-25 12:44:26 [WARNING] - Entering 'greet'
  > Hello, Jonathan!
2025-09-25 12:44:26 [WARNING] - Exiting 'greet'


In [86]:
greet('Jonathan')

2025-09-25 12:45:14 [WARNING] - Entering 'greet'
  > Hello, Jonathan!
2025-09-25 12:45:14 [WARNING] - Exiting 'greet'


## More Real World Examples

### Slowing Down Code With Optional Parameters

In [87]:
import functools
import time

def slow_down(_func=None, *, rate=1):
    """Sleep given amount of seconds before calling the function"""
    def decorator_slow_down(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            time.sleep(rate)
            return func(*args, **kwargs)
        return wrapper
    if _func is None:
        return decorator_slow_down
    else:
        return decorator_slow_down(_func)
        

@slow_down(rate=2)
def countdown(from_number: int):
    if from_number < 1:
        print('Liftoff!')
    else:
        print(from_number)
        countdown(from_number - 1)

countdown(3)

3
2
1
Liftoff!


### Creating Singletons

Python has a number of singletons, classes with only one instance:
- `True`, `False`, `None`

This allows for use of `is` to check for equality, or rather, identity. We can use a decorator to store the first instance of a class as a function attribute, enforcing the singleton design pattern:

In [88]:
import functools

def singleton(cls):
    """Make a class a Singleton class"""
    @functools.wraps(cls)
    def wrapper_singleton(*args, **kwargs):
        if wrapper_singleton.instance is None:
            wrapper_singleton.instance = cls(*args, **kwargs)
        return wrapper_singleton.instance
    wrapper_singleton.instance = None
    return wrapper_singleton

@singleton
class TheOne:
    pass

first_one = TheOne()

another_one = TheOne()
id(first_one)

1995139107200

In [90]:
id(another_one)

1995139107200

In [91]:
first_one is another_one

True

### Caching Return Values

In [94]:
import functools

def count_calls(func):
    @functools.wraps(func)
    def wrapper_count_calls(*args, **kwargs):
        wrapper_count_calls.num_calls += 1
        print(f'Call {wrapper_count_calls.num_calls} of {func.__name__}()')
        return func(*args, **kwargs)
    wrapper_count_calls.num_calls = 0
    return wrapper_count_calls
 
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 2) + fibonacci(num - 1)

fibonacci(10)

Call 1 of fibonacci()
Call 2 of fibonacci()
Call 3 of fibonacci()
Call 4 of fibonacci()
Call 5 of fibonacci()
Call 6 of fibonacci()
Call 7 of fibonacci()
Call 8 of fibonacci()
Call 9 of fibonacci()
Call 10 of fibonacci()
Call 11 of fibonacci()
Call 12 of fibonacci()
Call 13 of fibonacci()
Call 14 of fibonacci()
Call 15 of fibonacci()
Call 16 of fibonacci()
Call 17 of fibonacci()
Call 18 of fibonacci()
Call 19 of fibonacci()
Call 20 of fibonacci()
Call 21 of fibonacci()
Call 22 of fibonacci()
Call 23 of fibonacci()
Call 24 of fibonacci()
Call 25 of fibonacci()
Call 26 of fibonacci()
Call 27 of fibonacci()
Call 28 of fibonacci()
Call 29 of fibonacci()
Call 30 of fibonacci()
Call 31 of fibonacci()
Call 32 of fibonacci()
Call 33 of fibonacci()
Call 34 of fibonacci()
Call 35 of fibonacci()
Call 36 of fibonacci()
Call 37 of fibonacci()
Call 38 of fibonacci()
Call 39 of fibonacci()
Call 40 of fibonacci()
Call 41 of fibonacci()
Call 42 of fibonacci()
Call 43 of fibonacci()
Call 44 of fibonacci

55

In [95]:
def cache(func):
    """Keep a cache of previous function calls"""
    @functools.wraps(func)
    def wrapper_cache(*args, **kwargs):
        cache_key = args + tuple(kwargs.items())
        if cache_key not in wrapper_cache.cache:
            wrapper_cache.cache[cache_key] = func(*args, **kwargs)
        return wrapper_cache.cache[cache_key]
    wrapper_cache.cache = {}
    return wrapper_cache

@cache
@count_calls
def fibonacci(num):
    if num < 2:
        return num
    return fibonacci(num - 1) + fibonacci(num - 2)

fibonacci(10)

Call 1 of fibonacci()
Call 2 of fibonacci()
Call 3 of fibonacci()
Call 4 of fibonacci()
Call 5 of fibonacci()
Call 6 of fibonacci()
Call 7 of fibonacci()
Call 8 of fibonacci()
Call 9 of fibonacci()
Call 10 of fibonacci()
Call 11 of fibonacci()


55

In [96]:
fibonacci(8)

21

In [97]:
@functools.lru_cache(maxsize=4)
def fibonacci(num: int):
    if num < 2:
        value = num
    else:
        value = fibonacci(num - 1) + fibonacci(num - 2)
    print(f"Calculated fibonacci({num}) = {value}")
    return value


fibonacci(10)

Calculated fibonacci(1) = 1
Calculated fibonacci(0) = 0
Calculated fibonacci(2) = 1
Calculated fibonacci(3) = 2
Calculated fibonacci(4) = 3
Calculated fibonacci(5) = 5
Calculated fibonacci(6) = 8
Calculated fibonacci(7) = 13
Calculated fibonacci(8) = 21
Calculated fibonacci(9) = 34
Calculated fibonacci(10) = 55


55

In [98]:
fibonacci(8)

21

In [99]:
fibonacci(5)

Calculated fibonacci(1) = 1
Calculated fibonacci(0) = 0
Calculated fibonacci(2) = 1
Calculated fibonacci(3) = 2
Calculated fibonacci(4) = 3
Calculated fibonacci(5) = 5


5

In [100]:
fibonacci(8)

Calculated fibonacci(6) = 8
Calculated fibonacci(7) = 13
Calculated fibonacci(8) = 21


21

In [102]:
fibonacci(5)

5

In [103]:
fibonacci.cache_info()

CacheInfo(hits=17, misses=20, maxsize=4, currsize=4)

### Adding Information About Units

In [ ]:
def set_unit(unit):
    def set_unit_wrapper(func):
        func.unit = unit
        return func
    return set_unit_wrapper

## Getter and Setters

In [3]:
class Label:
    def __init__(self, text, font):
        self._text = text
        self._font = font

    def get_text(self):
        return self._text

    def set_text(self, value):
        self._text = value

    def get_font(self):
        return self._font

    def set_font(self, value):
        self._font = value

In [4]:
label = Label("Fruits", "JetBrains Mono NL")

In [9]:
label._text = 'hello'

In [10]:
label._text

'hello'

## Properties: Introducing Functionality to Attributes

These notes are derived from [this](https://realpython.com/python-getter-setter/) Real Python article. Attributes can be manipulated in two ways:
- directly
- via *getter* and *setter* methods (requires non-public API)


Rather than define our own getter/setter methods manually, we can use the `property` decorator.



The following example sets `name` and `birth_date` as properties, and assigns them `setter` methods

In [ ]:
from datetime import date

class Employee:
    def __init__(self, name, birth_date):
        self.name = name
        self.birth_date = birth_date

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value.upper()

    @property
    def birth_date(self):
        return self._birth_date

    @birth_date.setter
    def birth_date(self, value):
        self._birth_date = date.fromisoformat(value)

## Descriptors: Attributes with Attached Behaviors

We can add to the previous example a new attribute, `start_date` and associated getter/setter methods:

In [ ]:
from datetime import date

class Employee:
    def __init__(self, name, birth_date, start_date):
        self.name = name
        self.birth_date = birth_date
        self.start_date = start_date

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value.upper()

    @property
    def birth_date(self):
        return self._birth_date

    @birth_date.setter
    def birth_date(self, value):
        self._birth_date = date.fromisoformat(value)

    @property
    def start_date(self):
        return self._start_date

    @start_date.setter
    def start_date(self, value):
        self._start_date = date.fromisoformat(value)

This seems repetitive so we can refactor the code and set up a `Date()` object since our attributes are setup the same way:

In [ ]:
from datetime import date

class Date:
    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, instance, owner):
        return instance.__dict__[self._name]

    def __set__(self, instance, value):
        instance.__dict__[self._name] = date.fromisoformat(value)

class Employee:
    birth_date = Date()
    start_date = Date()

    def __init__(self, name, birth_date, start_date):
        self.name = name
        self.birth_date = birth_date
        self.start_date = start_date

    @property
    def name(self):
        return self._name

    @name.setter
    def name(self, value):
        self._name = value.upper()